# Préparation des données


In [2]:
# Import des package

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC


In [3]:
# Lecture du CSV
df = pd.read_csv('churn_train_bank.csv')
df.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       10000 non-null  int64  
 1   credit_score      10000 non-null  int64  
 2   country           10000 non-null  object 
 3   gender            10000 non-null  object 
 4   age               10000 non-null  int64  
 5   tenure            10000 non-null  int64  
 6   balance           10000 non-null  float64
 7   products_number   10000 non-null  int64  
 8   credit_card       10000 non-null  int64  
 9   active_member     10000 non-null  int64  
 10  estimated_salary  10000 non-null  float64
 11  churn             10000 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 937.6+ KB


Nous cherchons à effectuer des prédictions sur les clients en général. Nous pouvons supprimer customer_id et country

In [5]:
df.drop(['customer_id', 'country'], axis=1, inplace=True)

In [6]:
#Transformation de product_number en type object
df.products_number = df['products_number'].astype('object')

# Encodage de la variable gender
df['gender'] = LabelEncoder().fit_transform(df['gender'])



In [7]:
# Création df des varaibles explicatives
feats = df.drop('churn', axis=1)

# Cration de la varaible cible
target = df['churn']

In [8]:
# Transform products_number en plusieurs variables indicatrices
feats = pd.get_dummies(feats, prefix=['products'], prefix_sep='_')

In [9]:
# Création ensemble entrainement et test (75/25%)
X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size=0.25)

In [10]:
# Standardisation des variable continues

cols = ['credit_score','age', 'balance', 'estimated_salary']

X_train[cols]= StandardScaler().fit_transform(X_train[cols])
X_test[cols]= StandardScaler().fit_transform(X_test[cols])

In [11]:
# Affichage distribution target en % 
target.value_counts(normalize=True)

churn
0    0.7963
1    0.2037
Name: proportion, dtype: float64

Dans notre base cleints, 20.37% des clients présents ont quitté la banque.

# SVM simple

In [12]:
# Création modèle classification SVM 
svm = SVC(gamma='scale')

# Entrainement sur ensemble train
svm.fit(X_train, y_train)

# Score
print('Score sur l\'ensemble de test', svm.score(X_test, y_test))

Score sur l'ensemble de test 0.8392


Notre modèle obtient 83% des bonnes prédictions. 

Notre objectif est de prédire les départs éventuels des cleints. Est-ce que le résultat signifie que sur 10 clients churners 8 seront identifiées comme tel ? Non. 

Pour réussir à détecter le comportement naïf d’un modèle, l’outil le plus efficace est toujours la matrice de confusion. 

In [13]:
# Prédiction sur le modèle test
y_pred = svm.predict(X_test)

# Matrice de confusion

pd.crosstab(y_test, y_pred, colnames=['Predictions'])

Predictions,0,1
churn,,
0,1984,17
1,385,114


la matrice de confusion nous indique que le bon taux de bonnes prédictions obtenu est largement influencé par le bon comportement du modèle sur la classe dominante (0).

Le F1-score permet de mesurer la précision et le rappel à la fois.


Dans le cas d'une classification binaire, la sensibilité et la spécificité correspondent respectivement aux rappels de la classe positive et de la classe négative.


Une autre métrique, la moyenne géometrique (geometric mean), s'avère utile pour les problèmes de classification déséquilibrée : il s'agit de la racine du produit de la sensibilité et de la spécificité.


La fonction classification_report_imbalanced() permet d'afficher un rapport contenant notamment les résultats sur l'ensemble des métriques du package.

In [14]:
#Affichage rapport évlutaion du modèle sur test
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      0.99      0.23      0.91      0.48      0.24      2001
          1       0.87      0.23      0.99      0.36      0.48      0.21       499

avg / total       0.84      0.84      0.38      0.80      0.48      0.24      2500



Le tableau montre que le rappel et le f1-score de la classe 1 sont mauvais, tandis que pour la class 0, ils sont élevés. La moyenne géométrique est également faible. Le modèle n'est donc pas acceptable pour notre problème. 

# L'oversampling


Des méthodes dites de rééchantillonnage permettent de modifier les données avant d’entraîner le modèle dessus. Ces méthodes se divisent en 2 groupes principaux : les méthodes de sur-échantillonnage (Oversampling) et de sous-échantillonnage (Undersampling).

Nous allons aborder l'Oversampling aléatoire et le SMOTE.

Création de nouveau ensemble de donnée à partir de x et y train 

In [15]:
#-> Oversampling aléatoire
X_ro, y_ro = RandomOverSampler().fit_resample(X_train, y_train)

# Affichage nb élément
print('Classes échantillon oversampled:', dict(pd.Series(y_ro).value_counts()))



Classes échantillon oversampled: {0: 5962, 1: 5962}


In [16]:
#-> SMOTE
X_sm, y_sm = SMOTE().fit_resample(X_train, y_train)

# Affichage nb élément
print('Classes échantillon SMOTE :', dict(pd.Series(y_sm).value_counts()))

Classes échantillon SMOTE : {0: 5962, 1: 5962}


Entraînement le modèle sur l'ensemble issu du RandomOversampler

In [17]:
# Création modèle class SVM
svm = SVC(gamma='scale')

#Entrainement du modèle
svm.fit(X_ro, y_ro)

# Prédiction
y_pred = svm.predict(X_test)

# Matrice de confusion
pd.crosstab(y_test, y_pred)

col_0,0,1
churn,,
0,1547,454
1,124,375


In [19]:
# Classification report imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.77      0.75      0.84      0.76      0.58      2001
          1       0.45      0.75      0.77      0.56      0.76      0.58       499

avg / total       0.83      0.77      0.76      0.79      0.76      0.58      2500



La geometric mean est plus élévé. Le f1 est déjà meilleur que sur l’entrainement précédent. 

Entraînement le modèle sur l'ensemble issu du SMOTE

In [20]:
# Création modèle class SVM
svm = SVC(gamma='scale')

#Entrainement du modèle
svm.fit(X_sm, y_sm)

# Prédiction
y_pred = svm.predict(X_test)

# Matrice de confusion
pd.crosstab(y_test, y_pred)

col_0,0,1
churn,,
0,1573,428
1,141,358


In [21]:
# Classification report imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.79      0.72      0.85      0.75      0.57      2001
          1       0.46      0.72      0.79      0.56      0.75      0.56       499

avg / total       0.83      0.77      0.73      0.79      0.75      0.57      2500



L'entrainement le plus qualitaif est l'entrainement du modèle sur l'ensemble issu du RandomOversampler

# L'undersampling

Les méthodes d'Undersampling fonctionnent en diminuant le nombre d'observations de la / des classes majoritaires afin d'arriver à un ratio classe minoritaire / classe majoritaire satisfaisant.
Elles fonctionnent par sélection ou génération d'échantillons.


In [22]:
#Random Undersampling
rUs = RandomUnderSampler()
X_ru, y_ru = rUs.fit_resample(X_train, y_train)
print('Classes échantillon undersampled :', dict(pd.Series(y_ru).value_counts()))

#Centroids
cc = ClusterCentroids()
X_cc, y_cc = cc.fit_resample(X_train, y_train)
print('Classes échantillon CC :', dict(pd.Series(y_cc).value_counts()))

Classes échantillon undersampled : {0: 1538, 1: 1538}


c:\Users\sarah\miniconda3\envs\projetct\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Classes échantillon CC : {0: 1538, 1: 1538}


In [23]:
svm = SVC(gamma='scale')
svm.fit(X_ru, y_ru)

y_pred = svm.predict(X_test)
print(pd.crosstab(y_test, y_pred))
print(classification_report_imbalanced(y_test, y_pred))

col_0     0    1
churn           
0      1469  532
1       115  384
                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.73      0.77      0.82      0.75      0.56      2001
          1       0.42      0.77      0.73      0.54      0.75      0.57       499

avg / total       0.83      0.74      0.76      0.76      0.75      0.56      2500



In [24]:
svm = SVC(gamma='scale')
svm.fit(X_cc, y_cc)

y_pred = svm.predict(X_test)
print(pd.crosstab(y_test, y_pred))
print(classification_report_imbalanced(y_test, y_pred))

col_0     0    1
churn           
0      1208  793
1        96  403
                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.60      0.81      0.73      0.70      0.48      2001
          1       0.34      0.81      0.60      0.48      0.70      0.50       499

avg / total       0.81      0.64      0.77      0.68      0.70      0.48      2500



Toutes les méthodes appliquées ici donnent de meilleurs résultats que ceux obtenus par l'ensemble d'entraînement original.
Cependant, on peut remarquer qu'avec des résultats quasi identiques, les méthodes d'Undersampling permettent aux modèles d'être entraîné sur un échantillon de taille très réduite (environ quatre fois moins d'observations), ce qui se révèle utile pour de grosse bases de données.
Parfois, les méthodes de ré-échantillonnage ne sont pas assez efficaces, et dans ce cas il convient de repenser le problème.


# Autre méthode

il suffit d'ajouter dans notre modèle SVM par exemple l'argument probability=True, et d'utiliser la fonction predict_proba afin de retourner les probabilités d'appartenir à chacune des classes, au lieu de la classe la plus probable.


Si la probabilité d'appartenir à la classe 1 est supérieur au seuil défini, le client sera considéré comme un potentiel churner. Plus le seuil est bas, plus la précision augmentera, mais le rappel diminuera.


In [25]:
svm = SVC(probability=True, gamma='scale') # 'probability= True' est nécessaire pour retourner les probas
svm.fit(X_ru, y_ru)                        # mais ralentit l'entraînement

threshold = 0.5 # Tester avec 0.4, 0.6, ...

probs = svm.predict_proba(X_test)
pred_class = (probs[:,1]>=threshold).astype('int')

pd.crosstab(y_test, pred_class)

col_0,0,1
churn,,
0,1478,523
1,118,381


In [26]:
svm = SVC(probability=True, gamma='scale') # 'probability= True' est nécessaire pour retourner les probas
svm.fit(X_ru, y_ru)                        # mais ralentit l'entraînement

threshold = 0.4 

probs = svm.predict_proba(X_test)
pred_class = (probs[:,1]>=threshold).astype('int')

pd.crosstab(y_test, pred_class)

col_0,0,1
churn,,
0,1300,701
1,79,420


In [27]:
svm = SVC(probability=True, gamma='scale') # 'probability= True' est nécessaire pour retourner les probas
svm.fit(X_ru, y_ru)                        # mais ralentit l'entraînement

threshold = 0.6 

probs = svm.predict_proba(X_test)
pred_class = (probs[:,1]>=threshold).astype('int')

pd.crosstab(y_test, pred_class)

col_0,0,1
churn,,
0,1630,371
1,154,345


Une autre solution, plus facile, mais pas moins efficace dans certains cas est simplement d'utiliser le paramètre class_weight disponible dans la plupart des classes d'algorithmes de scikit-learn.


Il permet de pénaliser les erreurs faites sur une classe par un nouveau poids. Plus le poids d'une classe est important, plus les erreurs sur cette classe sont pénalisées, et plus on lui donne de l'importance.


Les poids doivent être indiqués sous forme de dictionnaire, par exemple : {0: 1, 1: 5}, pour donner 5 fois plus de poids aux erreurs faites sur la classe 1.
L'argument 'balanced' permet d'associer à chaque classe un poids inversement proportionnel à sa fréquence.

In [28]:
svm = SVC(gamma='scale', class_weight='balanced')
svm.fit(X_train, y_train)                         

preds = svm.predict(X_test)

pd.crosstab(y_test, preds)

col_0,0,1
churn,,
0,1509,492
1,119,380


Une dernière solution proposée par le sous-package imblearn.ensemble, est l'utilisation de classes contenant des modèles d'ensembles comme le Boosting ou Bagging et entraînés à chaque étape de l'algorithme sur un échantillon rééquilibré automatiquement entre les différentes classes.


Ces implémentations de modèles permettent de se passer de méthode de ré-échantillonnage avant l'entraînement, et de les appliquer de manière automatique à chaque sélection de données par l'algorithme.

In [29]:
from imblearn.ensemble import BalancedRandomForestClassifier

bclf = BalancedRandomForestClassifier()
bclf.fit(X_train, y_train) 
y_pred = bclf.predict(X_test)
pd.crosstab(y_test, y_pred)

c:\Users\sarah\miniconda3\envs\projetct\Lib\site-packages\imblearn\ensemble\_forest.py:546: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
c:\Users\sarah\miniconda3\envs\projetct\Lib\site-packages\imblearn\ensemble\_forest.py:558: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(


col_0,0,1
churn,,
0,1568,433
1,120,379


Tous nos modèles entraînés donnent de meilleurs résultats que ceux utilisant les ensembles de données initiaux.


Les méthodes vues s'appuient sur des procédés qui peuvent donner des résultats différents en fonction du type de données à notre disposition et de l'objectif poursuivi.


Il est important de retenir que plus le déséquilibre entre les classes est grand, moins les modèles classiques réussiront à prédire la classe minoritaire.


Dans de nombreux cas, les données réelles sont concernées par un problème de déséquilibre, il faudra donc nécessairement utiliser voire combiner certaines des méthodes.